<a href="https://colab.research.google.com/github/IsuruMahakumara/microsoft-ai-ml-engineering/blob/main/Practice_activity_Applying_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00


In [7]:
import peft

In [10]:
from transformers import BertForSequenceClassification

# Load a pre-trained BERT model for classification tasks
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Print model layers to identify attention layers where LoRA can be applied
for name, module in model.named_modules():
  if 'attention' in name:
    print(name)  # This output helps locate attention layers



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


bert.encoder.layer.0.attention
bert.encoder.layer.0.attention.self
bert.encoder.layer.0.attention.self.query
bert.encoder.layer.0.attention.self.key
bert.encoder.layer.0.attention.self.value
bert.encoder.layer.0.attention.self.dropout
bert.encoder.layer.0.attention.output
bert.encoder.layer.0.attention.output.dense
bert.encoder.layer.0.attention.output.LayerNorm
bert.encoder.layer.0.attention.output.dropout
bert.encoder.layer.1.attention
bert.encoder.layer.1.attention.self
bert.encoder.layer.1.attention.self.query
bert.encoder.layer.1.attention.self.key
bert.encoder.layer.1.attention.self.value
bert.encoder.layer.1.attention.self.dropout
bert.encoder.layer.1.attention.output
bert.encoder.layer.1.attention.output.dense
bert.encoder.layer.1.attention.output.LayerNorm
bert.encoder.layer.1.attention.output.dropout
bert.encoder.layer.2.attention
bert.encoder.layer.2.attention.self
bert.encoder.layer.2.attention.self.query
bert.encoder.layer.2.attention.self.key
bert.encoder.layer.2.attentio

In [11]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

# --- Mock Dataset Preparation (Replace with your actual data loading) ---

# 1. Load a raw dataset and tokenizer
raw_datasets = load_dataset("imdb")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# 2. Define tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True)

# 3. Create dummy splits and tokenize (similar to your previous steps)
# We will use the 'test' split for our mock validation set for simplicity.
tokenized_train = raw_datasets['train'].shuffle(seed=42).select(range(2000)).map(tokenize_function, batched=True)
tokenized_val = raw_datasets['test'].shuffle(seed=42).select(range(500)).map(tokenize_function, batched=True)

# 4. Remove unnecessary columns and rename 'label' (if needed)
tokenized_train = tokenized_train.remove_columns(["text"]).rename_column("label", "labels")
tokenized_val = tokenized_val.remove_columns(["text"]).rename_column("label", "labels")

# 5. Define Data Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

✅ Step 1: Mock Dataset Preparation Complete.


### Applying LoRA

In [12]:
from transformers import BertForSequenceClassification, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType

# 1. Load the base BERT model (for 3 labels: negative, neutral, positive)
num_labels = 3  # Assuming your task has 3 classes
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

# 2. Define the LoRA Configuration
lora_config = LoraConfig(
    # Specify the task type
    task_type=TaskType.SEQ_CLS,
    # Rank (r): Determines the rank of the update matrices. Lower r = smaller update, more efficient.  adjust_lora_rank(model, rank=2)
    r=8,
    # LoRA Alpha: Scaling factor for the weights. Usually set to 2*r.
    lora_alpha=16,
    # Dropout probability for the LoRA matrices
    lora_dropout=0.1,
    # target_modules: Modules in BERT to replace (common for attention layers)
    target_modules=["query", "key", "value"],
    bias="none",
)

# 3. Inject the LoRA adapters into the base model
lora_model = get_peft_model(model, lora_config)


# Display trainable parameters to show the huge reduction (only LoRA matrices B and A are trainable)
print("\n🔥 Trainable Parameters after LoRA Injection:")
lora_model.print_trainable_parameters()
print("✅ Step 2: LoRA Model Applied.")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🔥 Trainable Parameters after LoRA Injection:
trainable params: 444,675 || all params: 109,929,222 || trainable%: 0.4045
✅ Step 2: LoRA Model Applied.


### Fine-tune code

In [ ]:
from transformers import Trainer
import numpy as np
import evaluate

# 1. Define evaluation metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 2. Define Training Arguments
training_args = TrainingArguments(
    output_dir="./lora_results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",  # Disable wandb/other loggers
)

# 3. Initialize the Trainer
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 4. Start the training (Mock)
print("\n⏳ Step 3: Starting LoRA Fine-Tuning (mock run)...")
trainer.train()
print("✅ Step 3: Fine-Tuning Code Setup Complete.")

/tmp/ipython-input-816752553.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



⏳ Step 3: Starting LoRA Fine-Tuning (mock run)...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


### Evaluate

In [ ]:
# 1. Evaluate the final model
eval_results = trainer.evaluate()

# print("\n📊 Step 4: Evaluation Results")
# print(eval_results)

print("✅ Step 4: Evaluation Code Setup Complete.")